In [17]:
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, UpSampling2D, Conv2D, Flatten, Dense
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras import backend as K
from keras import initializers
import keras

import numpy as np
import matplotlib.pyplot as plt

from os import listdir
from os import system
import os
import random

import imageio

In [22]:
np.set_printoptions(suppress=True)

img_width, img_height = 31, 31

nb_epoch = 50
batch_size = 32

In [3]:
base_dir = '/scratch/image_datasets/1_for_learned_sift/ready'

train_data_dir      = base_dir + '/patches/train'
validation_data_dir = base_dir + '/patches/validation'
test_data_dir       = base_dir + '/patches/test'

train_descrs_dir      = base_dir + '/descriptors_angles/train'
validation_descrs_dir = base_dir + '/descriptors_angles/validation'
test_descrs_dir       = base_dir + '/descriptors_angles/test'

In [4]:
def loading_data(dir_patches, dir_descrs):
    files_patches = listdir(dir_patches + '/class0')
    files_patches.sort()
    
    files_descrs = listdir(dir_descrs + '/class0')
    files_descrs.sort()
    
    assert len(files_patches) == len(files_descrs), "The number of patches doesn't match the number of descriptors."

    patches = []
    descrs = []

    
    for file_patch, file_descr in zip(files_patches, files_descrs):
        patch = imageio.imread(dir_patches + '/class0/' + file_patch)
#         print(patch.shape)
        if patch.shape[0] == 31:
            patches.append(patch)
            descr = np.load(dir_descrs + '/class0/' + file_descr)
            descrs.append(descr)
#         elif image.shape[0] == 19:
#             temp_count19 += 1
        
        

    patches = np.array(patches)
    patches = patches.astype(np.float64) / 255
    
    descrs = np.array(descrs)
    descrs = descrs.astype(np.float64) / 255
    print("patches", patches.shape, "  descrs", descrs.shape)
    
    return patches, descrs

In [5]:
x_train, y_train = loading_data(train_data_dir, train_descrs_dir)
x_validation, y_validation = loading_data(validation_data_dir, validation_descrs_dir)

patches (3382, 31, 31)   descrs (3382, 129)
patches (409, 31, 31)   descrs (409, 129)


In [6]:
x_train = x_train.reshape((x_train.shape[0], x_train.shape[2], x_train.shape[2], 1))
x_validation = x_validation.reshape((x_validation.shape[0], x_validation.shape[2], x_validation.shape[2], 1))

In [7]:
x_validation.shape

(409, 31, 31, 1)

In [15]:
last_layer_bias = np.median(y_train)

In [27]:
last_layer_bias

0.058823529411764705

In [26]:
def MSLE_plus_plus(y_true, y_pred):
    if not K.is_tensor(y_pred):
        y_pred = K.constant(y_pred)
    y_true = K.cast(y_true, y_pred.dtype)
    first_log = K.log(K.clip(y_pred, K.epsilon(), None) + 1.)
    second_log = K.log(K.clip(y_true, K.epsilon(), None) + 1.)
    custom_loss_log = K.log(K.clip(y_true + y_pred, K.epsilon(), None) + 1.)
    custom_loss_denominator = (y_true * y_pred + 0.005) * 256  # parameters to be further adjusted
    return K.mean(K.square(first_log - second_log) + custom_loss_log / custom_loss_denominator, axis=-1)

In [29]:
input_shape = (img_width, img_height, 1)
input_img = Input(shape=input_shape)

x = Conv2D(32, (3, 3), activation="elu", padding="valid")(input_img)
x = Conv2D(32, (3, 3), activation="elu", padding="valid")(x)
x = Conv2D(32, (3, 3), activation="elu", padding="valid")(x)
x = MaxPooling2D((2, 2), padding="valid")(x)
x = Conv2D(32, (3, 3), activation="elu", padding="valid")(x)
x = Conv2D(32, (3, 3), activation="elu", padding="valid")(x)
x = Conv2D(32, (3, 3), activation="elu", padding="valid")(x)
x = MaxPooling2D((2, 2), padding="valid")(x)
x = Flatten(data_format="channels_last")(x)
encoded = Dense(129, activation="elu", bias_initializer=initializers.Constant(last_layer_bias))(x)

encoder = Model(input_img, encoded)

encoder.compile(optimizer='adadelta', loss="binary_crossentropy")
encoder.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 31, 31, 1)         0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 29, 29, 32)        320       
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 27, 27, 32)        9248      
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 25, 25, 32)        9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 10, 10, 32)        9248      
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 8, 8, 32)          9248

In [19]:
def fixed_generator(x_train, y_train, batch_size):
    while True:
        batch_list_x = []
        batch_list_y = []
        
        for i in range(x_train.shape[0]):
            batch_list_x.append(x_train[i])
            batch_list_y.append(y_train[i])
            if len(batch_list_x) == batch_size:
                yield (np.array(batch_list_x),np.array(batch_list_y))
                batch_list_x = []
                batch_list_y = []


In [ ]:
# model_version = '2.0.0.0_encoder_lastsigmoid_bce_adadelta'
# encoder = load_model(base_dir + '/ae' + model_version + '.h5')

In [31]:
model_version = '2.0.0.4_encoder_elueverywhere_bce_adadelta_initbiasYmed_3c1p3c1pd'

os.system('mkdir ' + base_dir + '/weights' + model_version)
checkpointer = keras.callbacks.ModelCheckpoint(base_dir + '/weights' + model_version + '/weights.{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)

encoder.fit_generator(fixed_generator(x_train, y_train, batch_size),
                steps_per_epoch=x_train.shape[0],
                epochs=100,
                validation_data=fixed_generator(x_validation, y_validation, batch_size),
                validation_steps=x_validation.shape[0],
                callbacks=[checkpointer]
                )
encoder.save(base_dir + '/ae' + model_version + '.h5')

Epoch 1/100
3382/3382 [==============================] - 61s 18ms/step - loss: 0.3339 - val_loss: 0.3345
Epoch 2/100
3382/3382 [==============================] - 61s 18ms/step - loss: 0.3322 - val_loss: 0.3348
Epoch 3/100
3382/3382 [==============================] - 61s 18ms/step - loss: 0.3307 - val_loss: 0.3453
Epoch 4/100
3382/3382 [==============================] - 61s 18ms/step - loss: 0.3297 - val_loss: 0.3473
Epoch 5/100
3382/3382 [==============================] - 61s 18ms/step - loss: 0.3286 - val_loss: 0.3416
Epoch 6/100
3382/3382 [==============================] - 61s 18ms/step - loss: 0.3280 - val_loss: 0.3333
Epoch 7/100
3382/3382 [==============================] - 61s 18ms/step - loss: 0.3270 - val_loss: 0.3405
Epoch 8/100
3382/3382 [==============================] - 61s 18ms/step - loss: 0.3263 - val_loss: 0.3615
Epoch 9/100
3382/3382 [==============================] - 61s 18ms/step - loss: 0.3257 - val_loss: 0.3337
Epoch 10/100
3382/3382 [==============================]

3382/3382 [==============================] - 60s 18ms/step - loss: 0.3188 - val_loss: 0.3840
Epoch 79/100
3382/3382 [==============================] - 60s 18ms/step - loss: 0.3186 - val_loss: 0.3804
Epoch 80/100
3382/3382 [==============================] - 60s 18ms/step - loss: 0.3182 - val_loss: 0.4312
Epoch 81/100
3382/3382 [==============================] - 60s 18ms/step - loss: 0.3187 - val_loss: 0.3506
Epoch 82/100
3382/3382 [==============================] - 60s 18ms/step - loss: 0.3184 - val_loss: 0.4327
Epoch 83/100
3382/3382 [==============================] - 60s 18ms/step - loss: 0.3185 - val_loss: 0.4227
Epoch 84/100
3382/3382 [==============================] - 60s 18ms/step - loss: 0.3192 - val_loss: 0.4073
Epoch 85/100
3382/3382 [==============================] - 60s 18ms/step - loss: 0.3190 - val_loss: 0.3936
Epoch 86/100
3382/3382 [==============================] - 60s 18ms/step - loss: 0.3192 - val_loss: 0.3859
Epoch 87/100
3382/3382 [==============================] - 6

In [22]:
np.around(encoder.predict(imageio.imread(test_data_dir + "/class0/patch000013.bmp").reshape(1,31,31,1))*255)

array([[255., 255., 255.,   0.,   0.,   0., 255., 255., 255., 255., 195.,
          0.,   0.,   0., 255.,   0., 255., 255.,   0., 255.,   0.,   0.,
          0.,   0.,   0.,   0.,   0., 255., 255., 255.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0., 255., 255., 255., 255.,   0.,   0.,
          0.,   0., 255.,   0., 255.,   6.,   0.,   0., 255.,   0.,   0.,
          0.,   0.,   0.,   0., 255., 255.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0., 255., 255.,   0.,   0.,   0.,   0.,   0.,
          0., 255., 255., 255.,   0.,   0.,   0.,   0., 255., 255., 255.,
        255.,   0.,   0.,   0., 255., 255., 255., 255.,   0.,   0., 255.,
        255.,   0.,   0.,   0.,   0.,   0., 255., 255., 255.,   0.,   0.,
          0.,   0., 255., 255., 255.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0., 255.,   0.,   0., 255.,   0., 255.]], dtype=float32)

In [32]:
np.around(encoder.predict(imageio.imread(train_data_dir + "/class0/patch000001.bmp").reshape(1,31,31,1))*255)

array([[ 532.,  832.,  222.,  -11.,  375.,   12., -219., -204., 1102.,
         999.,  396.,  116.,  386.,  -95., -190., -198., 1321.,  842.,
         220.,  -10.,  -29., -208., -232., -159.,  361.,   93.,  -10.,
         109.,  134., -234., -246., -224.,  316.,  613.,  506.,  169.,
         213., -140., -244., -237., 1235., 1202.,  498.,  -14.,   -5.,
        -129., -214.,   25.,  646.,  407.,  122.,   -4.,  122., -177.,
        -225., -237.,  234.,  242.,  112.,  397.,  266., -247., -251.,
        -230.,  468.,  258.,  442.,  210., -162., -232., -250., -203.,
        1548.,  908.,  405., -168., -205., -239., -236.,  292.,  298.,
         -91.,  179.,  383.,  104.,  -93., -158.,  273.,   14., -158.,
        -142.,  250.,  295., -196., -180.,   64.,  989.,  356.,   67.,
        -174., -232., -232., -159.,  424., 1862.,  688., -218., -245.,
        -240., -163.,  105., 1209.,  797., -179., -236., -197.,  -54.,
         325.,  372.,  734.,   84., -231., -244., -149.,  278.,  514.,
      

In [24]:
np.load(train_descrs_dir + "/class0/descr_angle_000001.npy")

array([  4,   1,   0,   0, 137,  42,   1,   3,  55,   7,   0,   0,  71,
        32,   1,   3, 137,  39,   0,   0,   3,   2,   0,   5,  40,  10,
         1,  13,  81,   1,   0,   1,  11,   2,   0,   0, 137,  64,   1,
         1,  54,   2,   0,   0, 109,  23,   1,   4, 137,  33,   4,   2,
         8,   2,   0,   8,  40,  16,  12,  83,  83,   0,   0,   1,  27,
         9,   0,   1, 137,  58,   2,   0,  23,   0,   0,   0,  54,  62,
        17,  18, 137,  36,  12,   8,   5,  10,   9,  34,  10,  16,  22,
       137,  76,   2,   0,   0,  50,   8,   0,   1, 137,  57,   1,   1,
        26,   1,   0,   0,  31,  26,  18,  19,  19,   2,   4,   4,  10,
        10,  17,  47,   1,   1,   2,  27, 137,  25,   2,   3,  19],
      dtype=uint8)

In [33]:
temp_y_p1 = np.load(train_descrs_dir + "/class0/descr_angle_000001.npy")
temp_x_p1 = np.around(encoder.predict(imageio.imread(train_data_dir + "/class0/patch000001.bmp").reshape(1,31,31,1))*255)
temp_x_p1 = temp_x_p1.reshape(129)

In [34]:
np.corrcoef(temp_x_p1, temp_y_p1)[0,1]

0.1900397385062676

In [35]:
corr_coefs = []
for i in range(103):
    try:
        temp_y_p1 = np.load(train_descrs_dir + "/class0/descr_angle_" + str(i).zfill(6) + ".npy")
        temp_x_p1 = np.around(encoder.predict(imageio.imread(train_data_dir + "/class0/patch" + str(i).zfill(6) + ".bmp").reshape(1,31,31,1))*255)
        temp_x_p1 = temp_x_p1.reshape(129)
        corr_coefs.append(np.corrcoef(temp_x_p1, temp_y_p1)[0,1])
    except:
        continue
corr_coefs = np.array(corr_coefs)
print(np.median(corr_coefs))

0.2225113365009303


In [45]:
temp_y_prime = encoder.predict(imageio.imread("/home/niaki/Code/ImageNet/tiny-imagenet-200/tiny_train16/class0/patch000012.png").reshape(1,16,16,3))

In [46]:
temp_y = np.load("/home/niaki/Code/ImageNet/tiny-imagenet-200/tiny_sifts/tiny_train16/class0/patch000012.npy")

In [47]:
print(np.corrcoef(temp_y, temp_y_prime[0])[1,0], "\n")
for i in range(temp_y_prime.shape[1]):
    print('{:>12f}{:>12f}'.format(temp_y[i], temp_y_prime[0, i]))      
#     print(temp_y_prime[0, i], temp_y[i])

0.6393183138370047 

    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    

In [21]:
np.round(temp, decimals=9)

array([[0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 3.427e-06, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+0

In [14]:
import tensorflow as tf

print(tf.__version__)

1.14.0


In [19]:
tf.disable_v2_behavior()

W0122 20:00:43.071549 140376228677440 deprecation.py:323] From /scratch/tensorflow/lib/python3.6/site-packages/tensorflow/python/compat/v2_compat.py:61: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term


In [21]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)